# 디스커버 통계 산출

In [1]:
from matplotlib import font_manager
from matplotlib import rc
import pickle
import re
import numpy as np
import seaborn as sns

f_path = '/Library/Fonts/Arial Unicode.ttf'
font_manager.FontProperties(fname=f_path).get_name()

rc('font', family='Arial Unicode MS')

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from networkx.algorithms import bipartite, community
from networkx.algorithms.community import modularity, louvain_communities
import matplotlib.colors as mcolors
from collections import Counter
import pandas as pd


# 1. 로딩

In [3]:
with open('../local_data/dynamics_attached_sheet.pkl', 'rb') as file:
    result_based_on_today = pickle.load(file)
    result_based_on_yesterday = pickle.load(file)

In [4]:
testers = list(result_based_on_today.keys())

## 일부 이상한 결과 유도하는 사용자 제외

In [5]:
testers.remove('최병건')
testers.remove('최지원')

In [ ]:
print(testers)

In [7]:
def make_daily_mean_adjusted_df(source_df:pd.DataFrame, tester):
    '''
    trial_idx 별 평균을 구한 후, 이를 뺀 값으로 보정한 메트릭을 추가함
    *_adjusted로 추가함
    trial_idx > 0 인 경우에만 계산함
    '''
    df = source_df[tester].query('trial_idx > 0').copy()
    df = df.dropna(subset=['keywords']) # NaN 제거
    df['trial_pos'] = df.groupby('trial_idx').cumcount()

    # Calculate the mean for each trial_idx for precision, freshness, and satisfaction
    trial_idx_means = df.groupby('trial_idx')[['precision', 'freshness', 'satisfaction']].mean()

    # Merge the mean values back to the original dataset
    df = df.merge(trial_idx_means, on='trial_idx', suffixes=('', '_mean'))

    # Calculate the adjusted values by subtracting the mean values from the original values
    df['precision_adjusted'] = df['precision'] - df['precision_mean']
    df['freshness_adjusted'] = df['freshness'] - df['freshness_mean']
    df['satisfaction_adjusted'] = df['satisfaction'] - df['satisfaction_mean']

    return df

In [8]:
dic_df = {tester:make_daily_mean_adjusted_df(result_based_on_today, tester) for tester in testers}

In [9]:
metric_cols = ['precision', 'freshness', 'satisfaction']
metric_cols_adjusted = [col + '_adjusted' for col in metric_cols]

# 2. 분석: 개인에 노출된 카테고리 분포 비교

카테고리 분포간 거리를 잴 수 있는 Hellinger distance 이용

특징
- 0 이면 동일
- 1 이면 가장 상이함
- Symmetric 함 `hellinger_distance(a, b) == hellinger_distance(b, a)`

In [10]:
def hellinger_distance(p, q):
    """
    두 categorical distribution 간의 Hellinger distance를 계산합니다.
    
    :param p: 첫 번째 분포 (NumPy 배열)
    :param q: 두 번째 분포 (NumPy 배열)
    :return: Hellinger distance (0에서 1 사이의 값)
    """
    # 두 분포의 길이가 같은지 확인
    if len(p) != len(q):
        raise ValueError("두 분포의 길이가 같아야 합니다.")
    
    # 각 분포의 합이 1인지 확인 (허용 오차 1e-8)
    if not np.isclose(np.sum(p), 1, atol=1e-8) or not np.isclose(np.sum(q), 1, atol=1e-8):
        raise ValueError("각 분포의 합은 1이어야 합니다.")
    
    # Hellinger distance 계산
    return np.sqrt(0.5 * np.sum((np.sqrt(p) - np.sqrt(q))**2))

In [11]:
category_freqs = {}
category_appeared = set() 
for tester, df in dic_df.items():
    category_freqs[tester] = df.category.value_counts()
    category_appeared |= set(category_freqs[tester].index)

등장하는 총 카테고리 수: 91

In [ ]:
len(category_appeared)

In [13]:
zero_freq_category_indexes = pd.Series([0]*len(category_appeared), index=category_appeared).rename('category', inplace=True)

In [14]:
category_distributions_dic = {}
for tester in dic_df.keys():
    xx = pd.merge(zero_freq_category_indexes, category_freqs[tester], left_index=True, right_index=True, how='left').fillna(0)['category_y']+0.001
    category_distributions_dic[tester] = xx / xx.sum()

베이스라인: 모든 테스터에 등장한 카테고리를 모두 더해서 평균 분포를 삼음

In [15]:

baseline_dist = pd.concat([pd.merge(zero_freq_category_indexes, category_freqs[tester], left_index=True, right_index=True, how='left').fillna(0)['category_y'] for tester in dic_df.keys()], axis=1).sum(axis=1)+0.001
baseline_dist /= baseline_dist.sum()

In [16]:
n = len(testers)
distance_matrix = np.zeros((n+1, n+1))

for i in range(n):
    for j in range(i+1, n):
        distance_matrix[i, j] = distance_matrix[j, i] = hellinger_distance(category_distributions_dic[testers[i]], category_distributions_dic[testers[j]])
        
for i in range(n):
    distance_matrix[i,n] = distance_matrix[n,i] = hellinger_distance(category_distributions_dic[testers[i]], baseline_dist)

방법
- 각 테스터에게 추천된 뉴스 기사의 카테고리의 빈도수를 분포화 (일자 무시, 만족도 무시)
- 분포간 거리를 계산하여, 디스커버에 노출된 콘텐츠들이 얼마나 상이한지 측정
- 베이스라인 분포 (테스터에게 노출된 카테고리를 모두 aggregation 한 분포)

결론
- 8명의 테스터에 노출된 콘텐츠가 상이한 것으로 관측됨
  - 테스트간 분포 차이 > 테스트와 베이스라인 분포 차이

In [ ]:
plt.figure(figsize=(5, 4))
sns.heatmap(distance_matrix, annot=True, cmap='coolwarm', xticklabels=testers+['base'], yticklabels=testers+['base'])
plt.title('Hellinger Distance Matrix between Testers')
plt.xlabel('Testers')
plt.ylabel('Testers')
plt.tight_layout()
plt.show()

# 분석: 중복 추천된 기사 비중 분석

테스트 마다 일자별로 뉴스에 할당된 핵심 키워드를 기준으로 중복 조사
- 중복 조사 방법
  - jaccard similarity: 두 집합을 비교하는 방법 jaccard(A, B) = |A ∩ B| / |A ∪ B|
- 의미
  - 중복 추천된 기사가 많다면, 그날의 각 뉴스의 유사도 평균 값이 작을 것이다 -> 반대의 경우는 크다

## 핵심 키워드 기준

In [18]:
from itertools import combinations

def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def calculate_average_jaccard_similarity(group):
    # print(group['keywords'])
    keyword_sets = [set(keywords) for keywords in group['keywords']]
    if len(keyword_sets) < 2:
        return 0
    similarities = [jaccard_similarity(set1, set2) 
                    for set1, set2 in combinations(keyword_sets, 2)]
    return sum(similarities) / len(similarities)

results = []

for tester in testers:
    result = dic_df[tester].groupby('trial_idx').apply(calculate_average_jaccard_similarity).reset_index()
    result['tester'] = tester
    result.columns = ['trial_idx', 'avg_jaccard_similarity', 'tester']
    # print(result)
    results.append(result)

# print(results)

In [ ]:
jaccard_similarity_df = pd.concat(results)

# 시각화
plt.figure(figsize=(6, 4))
sns.lineplot(data=jaccard_similarity_df, x='trial_idx', y='avg_jaccard_similarity', hue='tester', marker='o')
plt.title('Trial vs Avg Jaccard Similarity by Tester')
plt.xlabel('Trial Index')
plt.ylabel('Average Jaccard Similarity')
plt.legend(title='Tester')
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.violinplot(x='tester', y='avg_jaccard_similarity', data=jaccard_similarity_df, palette="muted", inner="box")

plt.title('Distribution of Avg Jaccard Similarity by Tester')
plt.xlabel('Tester')
plt.ylabel('Average Jaccard Similarity')
plt.show()


## 커뮤니티 기준

In [21]:
from itertools import combinations

def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def calculate_average_jaccard_similarity(group):
    # print(group['keywords'])
    keyword_sets = [set(keywords) for keywords in group['community']]
    if len(keyword_sets) < 2:
        return 0
    similarities = [jaccard_similarity(set1, set2) 
                    for set1, set2 in combinations(keyword_sets, 2)]
    return sum(similarities) / len(similarities)

results = []

for tester in testers:
    result = dic_df[tester].groupby('trial_idx').apply(calculate_average_jaccard_similarity).reset_index()
    result['tester'] = tester
    result.columns = ['trial_idx', 'avg_jaccard_similarity', 'tester']
    # print(result)
    results.append(result)

# print(results)

In [ ]:
jaccard_similarity_df = pd.concat(results)

# 시각화
plt.figure(figsize=(6, 4))
sns.lineplot(data=jaccard_similarity_df, x='trial_idx', y='avg_jaccard_similarity', hue='tester', marker='o')
plt.title('Trial vs Avg Jaccard Similarity by Tester (Community)')
plt.xlabel('Trial Index')
plt.ylabel('Average Jaccard Similarity')
plt.legend(title='Tester')
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.violinplot(x='tester', y='avg_jaccard_similarity', data=jaccard_similarity_df, palette="muted", inner="box")

plt.title('Distribution of Avg Jaccard Similarity by Tester (Community)')
plt.xlabel('Tester')
plt.ylabel('Average Jaccard Similarity')
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
# sns.lineplot(x='tester', y='avg_jaccard_similarity', data=jaccard_similarity_df, )
sns.boxplot(x='tester', y='avg_jaccard_similarity', data=jaccard_similarity_df)

plt.title('Distribution of Avg Jaccard Similarity by Tester (Community)')
plt.xlabel('Tester')
plt.ylabel('Average Jaccard Similarity')
plt.show()


# 분석: trial_idx 별로 메트릭 비교

trial_idx == 7을 제외하면 각 메트릭이 거의 일정함


In [24]:

dfs = []
for tester, df in dic_df.items():
    df_ = df.copy()
    df_['tester'] = tester
    dfs.append(df_)

In [25]:
import matplotlib.pyplot as plt

# Concatenate all dataframes excluding the two testers
data_filtered = pd.concat(dfs)

# Select relevant columns for analysis
cleaned_data_excluded = data_filtered[['trial_idx', 'precision', 'freshness', 'satisfaction']].dropna()

# Group by trial_idx and calculate mean values for precision, freshness, and satisfaction
trial_analysis_excluded = cleaned_data_excluded.groupby('trial_idx').mean()[['precision', 'freshness', 'satisfaction']]

# # Plot the values for each trial_idx excluding the two testers
# fig1 = plt.figure(figsize=(6, 4))
# plt.plot(trial_analysis_excluded.index, trial_analysis_excluded['precision'], label='Precision', marker='o')
# plt.fill_between(trial_analysis_excluded.index, trial_analysis_excluded['precision'] - trial_analysis_excluded['precision'].std(), trial_analysis_excluded['precision'] + trial_analysis_excluded['precision'].std(), alpha=0.3)
# plt.title('Precision Over Trial Index (Excluding 최병건 & 최지원)')
# plt.xlabel('Trial Index')
# plt.ylabel('Average Precision')
# plt.legend()
# plt.grid(True)
# plt.show()

# fig2 = plt.figure(figsize=(6, 4))
# plt.plot(trial_analysis_excluded.index, trial_analysis_excluded['freshness'], label='Freshness', marker='o')
# plt.fill_between(trial_analysis_excluded.index, trial_analysis_excluded['freshness'] - trial_analysis_excluded['freshness'].std(), trial_analysis_excluded['freshness'] + trial_analysis_excluded['freshness'].std(), alpha=0.3)
# plt.title('Freshness Over Trial Index (Excluding 최병건 & 최지원)')
# plt.xlabel('Trial Index')
# plt.ylabel('Average Freshness')
# plt.legend()
# plt.grid(True)
# plt.show()

# fig3 = plt.figure(figsize=(6, 4))
# plt.plot(trial_analysis_excluded.index, trial_analysis_excluded['satisfaction'], label='Satisfaction', marker='o')
# plt.fill_between(trial_analysis_excluded.index, trial_analysis_excluded['satisfaction'] - trial_analysis_excluded['satisfaction'].std(), trial_analysis_excluded['satisfaction'] + trial_analysis_excluded['satisfaction'].std(), alpha=0.3)
# plt.title('Satisfaction Over Trial Index (Excluding 최병건 & 최지원)')
# plt.xlabel('Trial Index')
# plt.ylabel('Average Satisfaction')
# plt.legend()
# plt.grid(True)
# plt.show()

# plt.figure(figsize=(6, 6))

# # Plot precision
# plt.subplot(2, 1, 1)
# sns.lineplot(data=streak_data, x='streak_count', y='precision')
# plt.title('streak_count vs precision')
# plt.xlabel('streak_count')
# plt.ylabel('Precision')

# # Plot satisfaction
# plt.subplot(2, 1, 2)
# sns.lineplot(data=streak_data, x='streak_count', y='satisfaction')
# plt.title('streak_count vs satisfaction')
# plt.xlabel('streak_count')
# plt.ylabel('Satisfaction')

# plt.tight_layout()
# plt.show()


In [ ]:
plt.figure(figsize=(10, 4))

# Plot precision
plt.subplot(1, 3, 1)
sns.lineplot(data=cleaned_data_excluded, x='trial_idx', y='precision', errorbar="sd")
plt.title('Trial Index vs Precision')
plt.xlabel('Trial Index')
plt.ylabel('Precision')

# Plot freshness
plt.subplot(1, 3, 2)
sns.lineplot(data=cleaned_data_excluded, x='trial_idx', y='freshness', errorbar="sd")
plt.title('Trial Index vs Freshness')
plt.xlabel('Trial Index')
plt.ylabel('Freshness')

# Plot satisfaction
plt.subplot(1, 3, 3)
sns.lineplot(data=cleaned_data_excluded, x='trial_idx', y='satisfaction', errorbar="sd")
plt.title('Trial Index vs Satisfaction')
plt.xlabel('Trial Index')
plt.ylabel('Satisfaction')

plt.tight_layout()
plt.show()


# 분석: Community 와 Satisfaction 관계 조사

community 마다 만족도가 차이가 큼

상위

```text
(2.4.1. 글로벌 기업 동향, 8.3. 국내 축구)      8.750000
(2.4. 산업/기업, 2.6. 경제 정책)            8.727273
(3.3. 노동, 6.1. 아시아/호주)              8.666667
(1.2.1. 외교/국제관계, 2.4. 산업/기업)        8.666667
(1.2.1. 외교/국제관계, 1.4.4. 테러리즘)       8.333333
```

하위

```text
(7.5. 애니메이션/웹툰,)                                    3.000000
(3.1. 사건사고, 7.2. 방송/드라마, 7.3. 음악, 7.4. 스타/연예인)      2.760000
(2.1.1. 글로벌 금융시장, 7.2. 방송/드라마, 7.4. 스타/연예인)         2.727273
(4.8. 육아/교육, 7.4. 스타/연예인)                           2.666667
(8.8. e스포츠,)                                        2.333333
(13.1. 정치인,)                                        1.000000
```


6인 모두에게 등장하는 community로 제한시

```text
(2.6. 경제 정책,)          7.190476
(1.2.1. 외교/국제관계,)      6.285714
(2.1. 금융,)             6.222222
(1.4.2. 무기/방위산업,)      6.142857
(4.1. 건강,)             6.025806
(3.1. 사건사고,)           5.939759
(5.1. 모바일,)            5.769231
(3.3. 노동,)             5.702703
(2.3. 부동산,)            5.661538
(4.4. 여행/레저,)          5.493671
(4.9. 반려동물,)           5.350000
(8.4. 해외 축구,)          5.319444
(3.2. 교육,)             5.272727
(5.2. 인터넷/SNS,)        5.238095
(2.2. 증권/주식,)          5.210526
(4.2. 요리/맛집,)          5.102564
(4.5. 자동차,)            5.000000
(4.10. 취미/DIY,)        4.895833
(5.5. 과학 일반,)          4.857143
(1.1.2. 국회/입법부,)       4.846154
(4.3. 패션/뷰티,)          4.259740
(1.1.1. 청와대/행정부,)      4.259259
(4.11. 웨딩/결혼,)         3.818182
(5.8. 블록체인/가상화폐,)      3.500000
(7.2. 방송/드라마,)         3.405405
```

In [ ]:
# Select relevant columns for Community and Satisfaction analysis
community_satisfaction_data = data_filtered[['community', 'satisfaction']].dropna()

# Group by community and calculate mean satisfaction
community_satisfaction_mean = community_satisfaction_data.groupby('community').mean().sort_values(by='satisfaction', ascending=False)



# Plot the community-wise satisfaction mean
plt.figure(figsize=(12, 6))

community_satisfaction_mean.plot(kind='bar', legend=False)
plt.title('Average Satisfaction by Community')
plt.xlabel('Community')
plt.ylabel('Average Satisfaction')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:
# Separate top and bottom communities
top_communities = community_satisfaction_mean.head(5)
bottom_communities = community_satisfaction_mean.tail(6)

print(top_communities)
print(bottom_communities)

## 6인 모두에게 등장하는 카테고리만 남겨서 관찰

In [29]:
community_nusers = data_filtered.groupby('community')['tester'].nunique()
ii = community_nusers[community_nusers >=6]

In [ ]:
community_shown_to_everyone = community_satisfaction_mean.loc[ii.index].sort_values(by='satisfaction', ascending=False)

# Plot the community-wise satisfaction mean
plt.figure(figsize=(10, 2))

community_shown_to_everyone.plot(kind='bar', legend=False)
plt.title('Average Satisfaction by Community')
plt.xlabel('Community')
plt.ylabel('Average Satisfaction')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

print(community_shown_to_everyone)

----

# Streak_count가 높은 콘텐츠의 특징

연속일이 증가할 수록
- precision 감소
- satisfaction도 감소하나 7일짜리 일때 증가
- 신뢰구간이 넓어짐 (해당 샘플 수가 적어지므로, 불확실성 증가)

In [ ]:
# Filter data for rows where 'type' is 'persisting' and streak_count is available
streak_data = data_filtered[data_filtered['type'] == 'persisting'].dropna(subset=['streak_count', 'precision', 'freshness', 'satisfaction'])

# Plot the results for precision and satisfaction over streak_count
plt.figure(figsize=(6, 4))
sns.lineplot(data=streak_data, x='streak_count', y='precision', label='Precision', errorbar='ci')
sns.lineplot(data=streak_data, x='streak_count', y='satisfaction', label='Satisfaction', errorbar='ci')
plt.title('streak_count vs precision and satisfaction')
plt.xlabel('streak_count')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(8, 4))

# Plot precision
plt.subplot(1, 2, 1)
sns.lineplot(data=streak_data, x='streak_count', y='precision', errorbar='ci')
plt.title('streak_count vs precision')
plt.xlabel('streak_count')
plt.ylabel('Precision')

# Plot satisfaction
plt.subplot(1, 2, 2)
sns.lineplot(data=streak_data, x='streak_count', y='satisfaction', errorbar='ci')
plt.title('streak_count vs satisfaction')
plt.xlabel('streak_count')
plt.ylabel('Satisfaction')

plt.tight_layout()
plt.show()

In [ ]:
# Group data by streak_count, community and calculate mean satisfaction for streak count 6 and 7
streak_6_communities = streak_data[streak_data['streak_count'] == 6].groupby('community')[['satisfaction']].mean().reset_index()
streak_7_communities = streak_data[streak_data['streak_count'] == 7].groupby('community')[['satisfaction']].mean().reset_index()

# Merge the two dataframes to show communities for streak count 6 and 7 along with their satisfaction
merged_streak_communities = pd.merge(streak_6_communities, streak_7_communities, on='community', suffixes=('_streak_6', '_streak_7'), how='outer')

merged_streak_communities


## 7일 동안 연속된 community의 precision, satisfaction 조사

In [44]:
filtered_streak_data = streak_data[streak_data['community'].isin(streak_7_communities.community)]


In [ ]:
# Plot the results for precision and satisfaction over streak_count
plt.figure(figsize=(6, 4))
sns.lineplot(data=filtered_streak_data, x='streak_count', y='precision', label='Precision')
sns.lineplot(data=filtered_streak_data, x='streak_count', y='satisfaction', label='Satisfaction')
plt.title('streak_count vs precision and satisfaction (7일 연속된 커뮤니티만)')
plt.xlabel('streak_count')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
# Plot the results for precision and satisfaction over streak_count
plt.figure(figsize=(6, 4))
sns.lineplot(data=filtered_streak_data, x='streak_count', y='precision', hue='community', errorbar=None)
plt.title('streak_count vs precision (7일 연속된 커뮤니티만)')
plt.xlabel('streak_count')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
# Plot the results for precision and satisfaction over streak_count
plt.figure(figsize=(6, 4))
sns.lineplot(data=filtered_streak_data, x='streak_count', y='satisfaction', hue='community', errorbar=None)
plt.title('streak_count vs satisfaction (7일 연속된 커뮤니티만)')
plt.xlabel('streak_count')
plt.ylabel('Value')
plt.legend()
plt.show()

## 블록 체인의 경우만 별도로

6일 연속해서 노출된 경우가 있음

In [ ]:
# Filter data for the community ('5.8. 블록체인/가상화폐',) and group by streak_count
blockchain_community_data = streak_data[streak_data['community'] == ('5.8. 블록체인/가상화폐',)]

# Plot precision
plt.figure(figsize=(6, 4))
sns.lineplot(data=blockchain_community_data, x='streak_count', y='precision', errorbar=None)
plt.title('streak_count vs precision')
plt.xlabel('streak_count')
plt.ylabel('Satisfaction')

plt.tight_layout()
plt.show()


In [ ]:
# Filter data for the community ('5.8. 블록체인/가상화폐',) and group by streak_count
blockchain_community_data = streak_data[streak_data['community'] == ('5.8. 블록체인/가상화폐',)]

# Plot satisfaction
plt.figure(figsize=(6, 4))
sns.lineplot(data=blockchain_community_data, x='streak_count', y='satisfaction', errorbar=None)
plt.title('streak_count vs satisfaction')
plt.xlabel('streak_count')
plt.ylabel('Satisfaction')

plt.tight_layout()
plt.show()


# 콘텐츠 유형별에 따른 Precision, Freshness, Satisfaction의 차이 분석

아주 선명하진 않지만
- new 의 경우 freshness 가 상대적으로 높음
- shrinking의 경우 precision이 상대적으로 높음 (관심사가 좁혀져서?)


In [ ]:
# Group data by 'type' and calculate mean values for precision, freshness, and satisfaction
type_analysis = data_filtered.groupby('type')[['precision', 'freshness', 'satisfaction']].mean()

# Plot the results for each type
plt.figure(figsize=(6, 4))

# Plot precision, freshness, and satisfaction for each type
type_analysis.plot(kind='bar', figsize=(10, 6), legend=True)
plt.title('Precision, Freshness, and Satisfaction by Content Type')
plt.xlabel('Content Type')
plt.ylabel('Average Value')
plt.xticks(rotation=45)
plt.ylim(4, None)  # Set the minimum value of the y-axis to 4
plt.grid(True)
plt.show()


---


# 분석: 같은 커뮤니티 내 만족도 차이 vs 다른 커뮤니티 끼리 만족도 차이가 다른지

인덱스 초기화 필요

In [66]:
data_filtered.reset_index(drop=True, inplace=True)

In [67]:
# Initialize a list to store satisfaction differences for the same community
satisfaction_differences = []

# Group by trial_idx for analysis
grouped_trials = data_filtered.groupby('tester')

# Iterate over each trial and calculate satisfaction differences for the same community
for tester, trial_data in grouped_trials:
    for (i, j) in combinations(trial_data.index, 2):
        if trial_data.loc[i, 'community'] == trial_data.loc[j, 'community']:
            satisfaction_diff = abs(trial_data.loc[i, 'satisfaction'] - trial_data.loc[j, 'satisfaction'])
            satisfaction_differences.append({
                'tester': tester,
                'satisfaction_diff': satisfaction_diff
            })

# Convert the results to a DataFrame for analysis and visualization
satisfaction_diff_df = pd.DataFrame(satisfaction_differences)

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(data=satisfaction_diff_df, x='tester', y='satisfaction_diff')
plt.title('Satisfaction Differences Distribution by tester (Same Community)')
plt.xlabel('Trial Index')
plt.ylabel('Satisfaction Difference')
plt.show()


In [ ]:
satisfaction_diff_df.groupby('tester').describe()

In [70]:
# Initialize a list to store satisfaction differences between different communities
satisfaction_differences = []

# Group by trial_idx for analysis
grouped_trials = data_filtered.groupby('tester')

# Iterate over each trial and calculate satisfaction differences between different communities
for tester, trial_data in grouped_trials:
    for (i, j) in combinations(trial_data.index, 2):
        if trial_data.loc[i, 'community'] != trial_data.loc[j, 'community']:
            satisfaction_diff = abs(trial_data.loc[i, 'satisfaction'] - trial_data.loc[j, 'satisfaction'])
            satisfaction_differences.append({
                'tester': tester,
                'satisfaction_diff': satisfaction_diff
            })

# Convert the results to a DataFrame for analysis and visualization
inter_satisfaction_diff_df = pd.DataFrame(satisfaction_differences)

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(data=inter_satisfaction_diff_df, x='tester', y='satisfaction_diff')
plt.title('Satisfaction Differences Distribution by tester (Same Community)')
plt.xlabel('Trial Index')
plt.ylabel('Satisfaction Difference')
plt.show()


In [ ]:
inter_satisfaction_diff_df.groupby('tester').describe()

---



In [ ]:
data_filtered.groupby(['tester'])[['precision', 'freshness', 'satisfaction']].describe().T